<a href="https://colab.research.google.com/github/asterray03/asterray03/blob/main/AI_Based_Communication_for_Military.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import networkx as nx
import numpy as np
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from sklearn.ensemble import IsolationForest

# === 🔹 Step 1: Key Exchange using ECDH (Elliptic Curve Diffie-Hellman) === #
def generate_keys():
    private_key = ec.generate_private_key(ec.SECP256R1(), default_backend())
    public_key = private_key.public_key()
    return private_key, public_key

def derive_shared_key(private_key, peer_public_key):
    shared_key = private_key.exchange(ec.ECDH(), peer_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'secure-comm',
        backend=default_backend()
    ).derive(shared_key)
    return derived_key

# === 🔹 Step 2: AES Encryption & Decryption with PKCS7 Padding === #
def encrypt_message(key, message):
    iv = os.urandom(16)  # Secure IV
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # ✅ Proper PKCS7 Padding
    padder = padding.PKCS7(128).padder()
    padded_message = padder.update(message.encode()) + padder.finalize()

    encrypted_message = encryptor.update(padded_message) + encryptor.finalize()
    return iv + encrypted_message  # Prepend IV for decryption

def decrypt_message(key, encrypted_message):
    iv = encrypted_message[:16]  # Extract IV
    encrypted_message = encrypted_message[16:]  # Extract actual encrypted data
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    decrypted_padded_message = decryptor.update(encrypted_message) + decryptor.finalize()

    # ✅ Remove PKCS7 Padding
    unpadder = padding.PKCS7(128).unpadder()
    decrypted_message = unpadder.update(decrypted_padded_message) + unpadder.finalize()

    return decrypted_message.decode()

# === 🔹 Step 3: AI-Powered Anomaly Detection for Cybersecurity === #
def train_anomaly_detector():
    np.random.seed(42)
    normal_data = np.random.normal(50, 10, (100, 2))  # Simulated normal network behavior
    attack_data = np.random.uniform(0, 100, (5, 2))   # Simulated attack behavior
    data = np.vstack((normal_data, attack_data))

    labels = np.array([1] * 100 + [-1] * 5)  # Normal = 1, Attack = -1
    model = IsolationForest(contamination=0.05)
    model.fit(data)

    accuracy = model.score_samples(data).mean()
    print(f"🔍 Anomaly Detection Model Accuracy: {accuracy * 100:.2f}%")
    return model

def detect_anomalies(model, new_data):
    prediction = model.predict([new_data])
    return "⚠️ Alert: Suspicious Activity Detected!" if prediction[0] == -1 else "✅ Secure Communication"

# === 🔹 Step 4: Multi-Hop Ad Hoc Networking for Battlefield === #
def create_battlefield_network():
    G = nx.Graph()
    nodes = ["HQ", "Soldier A", "Soldier B", "Drone", "Tank"]
    edges = [("HQ", "Soldier A"), ("Soldier A", "Soldier B"), ("Soldier B", "Drone"), ("Drone", "Tank")]
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)

    return G

def find_optimal_path(G, source, destination):
    return nx.shortest_path(G, source=source, target=destination)

# === 🔹 Step 5: Execute Secure Communication === #
if __name__ == "__main__":
    # ✅ Generate Key Pairs
    hq_private, hq_public = generate_keys()
    soldier_private, soldier_public = generate_keys()

    # ✅ Secure Key Exchange
    shared_key = derive_shared_key(hq_private, soldier_public)
    print("🔑 Shared Encryption Key Derived Successfully!")

    # ✅ Secure Message Transmission
    original_message = "Mission Update: Move to Grid 34"
    encrypted_message = encrypt_message(shared_key, original_message)
    decrypted_message = decrypt_message(shared_key, encrypted_message)

    print(f"📩 Encrypted Message: {encrypted_message.hex()}")
    print(f"📨 Decrypted Message: {decrypted_message}")

    # ✅ AI Cybersecurity Check
    model = train_anomaly_detector()
    new_data = [48, 52]  # Simulated real-time network data
    security_status = detect_anomalies(model, new_data)
    print(f"🔒 Security Check: {security_status}")

    # ✅ Multi-Hop Network Simulation
    battlefield_network = create_battlefield_network()
    optimal_path = find_optimal_path(battlefield_network, "HQ", "Tank")
    print(f"📡 Optimal Communication Path: {' ➝ '.join(optimal_path)}")


🔑 Shared Encryption Key Derived Successfully!
📩 Encrypted Message: 9c716abd2f696c7f06b2957a13ebb56337b687aaed8664226dac589823133d722cc02925e9aa662348cba0ac0ec39ad4
📨 Decrypted Message: Mission Update: Move to Grid 34
🔍 Anomaly Detection Model Accuracy: -46.21%
🔒 Security Check: ✅ Secure Communication
📡 Optimal Communication Path: HQ ➝ Soldier A ➝ Soldier B ➝ Drone ➝ Tank
